# Reading CSV data

In [3]:
import csv
EDGES = []
with open('vk_links.csv') as csv_file:
    reader = csv.reader(csv_file)
    skip_first = 1
    for row in reader:
        if skip_first == 1:
            skip_first = 0
            continue
        EDGES.append((row[0],row[1]))
    
# Getting unique names
NAMES = set([x for t in EDGES for x in t])

# Find out clusters in graph

In [4]:
import louvain      # pacaur -S igraph && sudo pip3 install louvain
import igraph as ig


# Creating igraph based on our data
graph = ig.Graph(directed=True)

# Adding vertices
for key in NAMES:
    graph.add_vertices(key)
    
# Adding edges
graph.add_edges(EDGES)

# Clustering data
partitions = louvain.find_partition(graph, louvain.ModularityVertexPartition)
#print(partitions)
society = []
for idx in range(len(partitions)):
    society.append(partitions.subgraph(idx))


# Rank users in societies

In [5]:
import numpy as np
from scipy.sparse import csc_matrix

def page_rank(graph, s=0.86, maxerr=0.001):
    """
    Page Rank algorithm for numpy adjacent matrix.
    
    
    """
    n = graph.shape[0]

    A = csc_matrix(graph,dtype=np.float)
    rsums = np.array(A.sum(1))[:,0]
    ri, ci = A.nonzero()
    A.data /= rsums[ri]

    sink = (rsums == 0)

    ro, r = np.zeros(n), np.ones(n)
    while (np.sum(np.abs(r-ro)) > maxerr):
        print("%d" % (np.sum(np.abs(r-ro))))
        ro = r.copy()
        # calculate each pagerank at a time
        for i in range(0,n):
            # inlinks of state i
            Ai = np.array(A[:,i].todense())[:,0]
            # account for sink states
            Di = sink / float(n)
            # account for teleportation to state i
            Ei = np.ones(n) / float(n)

            r[i] = ro.dot( Ai*s + Di*s + Ei*(1-s) )

    # return normalized pagerank
    return r/float(sum(r))

def label_members(vs, ranks, k=0.2):
    """
    Label members from `vs' using `ranks' with accuracy `k'
    
    We get maximum values from ranks and label the accurate winners as "leaders".
    The accurate loosers are "followers".
    All the other are "agents".
    """
    max_val = ranks[0]
    min_val = ranks[0]
    max_idx = min_idx = 0
    nodes = {}
    for idx, val in enumerate(ranks): 
        max_val = max(val, max_val)
        if max_val == val: max_idx = idx
        min_val = min(val, min_val)
        if min_val == val: min_idx = idx
        nodes[vs[idx]] = val
           
    difference = (max_val - min_val) * k
    labels = {}
    for node, rnk in nodes.items():
        if rnk == max_val:
            labels[node['name']] = 'leader'
        elif rnk == min_val:
            labels[node['name']] = 'follower'
        else:
            labels[node['name']] = 'agent'
            
    return labels

LABELED_ITEMS = {}
for (idx, cluster) in enumerate(society):
    adj = list(cluster.get_adjacency())
    ranks = page_rank(np.array(adj), s=0.85, maxerr=30)
    labels = label_members(cluster.vs, ranks, k=0.5) # less is not relevant, more is too common for this dataset
    #print(dir(cluster))
    LABELED_ITEMS[idx] = {
        'labels': {},
        'edges': list(map(lambda edge: (cluster.vs[edge[0]]['name'], cluster.vs[edge[1]]['name']),
                          cluster.get_edgelist())),
    }
    for person, label in labels.items():
        LABELED_ITEMS[idx]['labels'][person] = label

191
63
0.015380135778341911
0.0026929753908641794
112
38
0.023364706807022684
0.003927130524941232
79
0.026529154364601813
0.003932491115084508
69
0.039766762642120174
0.003372192304690415
40
0.06158385401652703
0.009247596528491317


# Plotting

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random

def rand_color(seed=90):
    random.seed(seed)
    r = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % (r(),r(),r())

G = nx.DiGraph()

names = list(NAMES)
for v in NAMES:
    G.add_node(names.index(v))

pos = nx.spring_layout(G)

for idx, items in LABELED_ITEMS.items():
    # add edges
    color = rand_color(idx**3)
    G.add_edges_from(list(map(lambda it: (names.index(it[0]),names.index(it[1])), items['edges'])))

    
    nx.draw_networkx_edges(G,
                           edgelist=list(map(lambda it: (names.index(it[0]),names.index(it[1])), items['edges'])),
                           pos=pos,
                           #arrowstyle='->',
                           arrowsize=20,
                           edge_color=color)
    
    nx.draw_networkx_nodes(G, 
                           pos, 
                           nodelist=[names.index(v) for v in items['labels'].keys()], 
                           node_color=color)
    
    nx.draw_networkx_labels(G, 
                            pos=pos,
                            labels={names.index(k): k+'\n'+v for k, v in items['labels'].items()},
                            node_color=color, 
                            font_size=13,
                            node_size=5000)

#print(list(map(lambda v: v['edges'], LABELED_ITEMS.values())))
gray_edges = []
labeled = list(map(lambda v: v['edges'], LABELED_ITEMS.values()))
all_labeled = [x for y in labeled for x in y]
for edge in EDGES:
    if not edge in all_labeled:
        gray_edges.append(edge)

#print(gray_edges)
nx.draw_networkx_edges(G,
                       edgelist=list(map(lambda it: (names.index(it[0]),names.index(it[1])), gray_edges)),
                       pos=pos,
                       alpha=0.5,
                       edge_color='#dddddd')
    
fig = plt.gcf()
ig.plot(G)
fig.set_size_inches(18.5, 10.5)
plt.figure(figsize=(20, 20), dpi=800)
plt.show()

# Writing to file

In [ ]:
with open('out.csv', mode='w') as out_file:
    writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['User', 'Community_ID', 'User_type'])
    for group, items in LABELED_ITEMS.items():
        for name, role in items['labels'].items():
            writer.writerow([name, group, role])
print('Done!')